# Food Bank Problem

In [42]:
import sys
import numpy as np
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization

## OPT - Waterfilling

In [43]:
## Water-filling Algorithm for sorted demands
def waterfilling_sorted(d,b):
    n = np.size(d)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        equal_allocation = bundle_remaining/(n-i)
        if d[i]<equal_allocation:
            allocations[i] = bundle_remaining if i==n-1 else d[i]
        else:
            allocations[i] = equal_allocation
        bundle_remaining -= allocations[i]
    return allocations

In [44]:
## Water-filling Algorithm for general demands
def waterfilling(d,b):
    n = np.size(d)
    sorted_indices = np.argsort(d)
    sorted_demands = np.sort(d)
    sorted_allocations = waterfilling_sorted(sorted_demands, b)
    allocations = np.zeros(n)
    for i in range(n):
        allocations[sorted_indices[i]] = sorted_allocations[i]
    return allocations

In [45]:
## Tests
assert list(waterfilling(np.zeros(0), 5)) == []
assert list(waterfilling(np.array([1,2,3,4]), 10)) == [1,2,3,4]
assert list(waterfilling(np.array([3,4,1,2]), 10)) == [3,4,1,2]
assert list(waterfilling(np.array([1,2,3,4]), 8)) == [1,2,2.5,2.5]
assert list(waterfilling(np.array([3,1,4,2]), 8)) == [2.5,1,2.5,2]
assert list(waterfilling(np.array([3,6,5,6]), 8)) == [2,2,2,2]

## Online Algorithms

In [46]:
## Online Water-filling taking minimum of realized demand and predetermeined allocation
def waterfilling_online_1(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    prior_allocations_assignment = waterfilling(demands_predicted,b)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = min(prior_allocations_assignment[i], demands_realized[i]) if i!=n-1 else bundle_remaining
        bundle_remaining -= allocations[i]
    return allocations

In [47]:
## Tests
assert list(waterfilling_online_1(np.zeros(0), np.zeros(0), 5)) == []
assert list(waterfilling_online_1(np.array([1,2,3,4]), np.array([5,5,5,5]), 10)) == [1,2,3,4]
assert list(waterfilling_online_1(np.array([3,1,4,2]), np.array([2,3,2.5,1]), 8)) == [2,1,2.5,2.5]


In [48]:
## Online Water-filling algorithm where each agent solves waterfilling with realized current demand and expected following demands
def waterfilling_online_2(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        allocations[i] = waterfilling(np.append(demands_realized[i],demands_predicted[i+1:]), bundle_remaining)[0]
        bundle_remaining -= allocations[i]
    return allocations

In [78]:
def insert_sorted(lst, element):
    n = np.size(lst)
    if n==0:
        return np.array([element]),0
    if element<lst[0]:
        return np.append(element,lst),0
    if element>lst[n-1]:
        return np.append(lst,element),n
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        if element == lst[mid_ind] or (element>lst[mid_ind] and element<lst[mid_ind+1]):
            return np.append(np.append(lst[:mid_ind+1],element),lst[mid_ind+1:]), mid_ind+1
    return np.append(np.append(lst[:left],element),lst[left:]), left

In [79]:
def delete_sorted(lst,element):
    n = np.size(lst)
    if element==lst[0]:
        return lst[1:]
    if element==lst[n-1]:
        return lst[:-1]
    left = 0
    right = n-1
    while left<right-1:
        mid_ind = int((left+right)/2)
        if element<lst[mid_ind]:
            right = mid_ind
        elif element > lst[mid_ind] :
            left = mid_ind
        else:
            return np.append(lst[:mid_ind],lst[mid_ind+1:])

In [80]:
## O(n^2) version of online algorithm that needs waterfilling evaluated multiple times
def waterfilling_dynamic(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    sorted_demands = np.sort(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        sorted_demands = delete_sorted(sorted_demands, demands_predicted[i])
        new_sorted_list,index = insert_sorted(sorted_demands,demands_realized[i])
        allocations[i] = (waterfilling_sorted(new_sorted_list, bundle_remaining))[index]
        bundle_remaining -= allocations[i]
    return allocations
    

In [81]:
## Tests 
assert list(waterfilling_online_2(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_2(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_2(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

assert list(waterfilling_dynamic(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_dynamic(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_dynamic(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]


In [82]:
## Online Water-filling algorithm where each agent is assigned infinite demand while finding optimal solution and allocation is readjusted
def waterfilling_online_3(demands_predicted, demands_realized, b):
    n = np.size(demands_predicted)
    allocations = np.zeros(n)
    bundle_remaining = b
    for i in range(n):
        future_allocations = waterfilling(np.append(np.Inf,demands_predicted[i+1:]), bundle_remaining)
        if future_allocations[0]>demands_realized[i] and i!=n-1:
            allocations[i] = demands_realized[i]
        else:
            allocations[i] = future_allocations[0]
        bundle_remaining -= allocations[i]
    return allocations

In [83]:
## Tests 
assert list(waterfilling_online_3(np.zeros(0), np.zeros(0), 5)) == []
assert list(np.around(waterfilling_online_3(np.array([1,2,3,4]), np.array([5,5,5,5]), 11),2)) == [3,2.67,2.67,2.67]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([2,1,8,6]), 15)) == [2,1,6,6]
assert list(waterfilling_online_3(np.array([4,5,3,6]), np.array([9,10,2,1]), 15)) == [4,4,2,5]

## Objective Function

In [84]:
## Calculate log of Nash welfare for objective function
def objective(demands, allocation):
    welfare_sum = 0
    for i in range(np.size(demands)):
        welfare_sum += np.log(min(1,allocation[i]/demands[i]))
    return welfare_sum

## Experiment

In [85]:
def make_demands_uniform_distribution(num_towns, demand_ranges):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.uniform(0, demand_ranges[i])
        expected_demands[i] = demand_ranges[i]/2
    return demands, expected_demands


In [86]:
def make_demands_exponential_distribution(num_towns, demand_means):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.exponential(demand_means[i])
        expected_demands[i] = demand_means[i]
    return demands, expected_demands


In [87]:
num_iterations = 10
num_towns_range = 100
demands_max = 20
max_n = 1000
fix_num_towns = 10
max_b = 200


### Regret for Uniform Distribution of Demands

In [88]:
data_dict_1 = {'NumTowns':[],'Alg_1':[],'Alg_3':[]}

for n in range(max_n):
    for i in range(num_iterations):
        data_dict_1['NumTowns'].append(n)
        demand_ranges = np.random.uniform(0, demands_max, n)
        town_demands, town_expected_demands = make_demands_uniform_distribution(n, demand_ranges)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict_1['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
#             if j==1:
#                 data_dict['Alg_2'].append(opt - objective(town_demands, waterfilling_online_1(town_expected_demands,town_demands,budget)))
            if j==2:
                data_dict_1['Alg_3'].append(opt - objective(town_demands, waterfilling_dynamic(town_expected_demands,town_demands,budget)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumTowns")
fig = px.scatter(df_uniform, x="NumTowns", y="value", color='variable')
fig.show()

### Regret for Exponential Distribution of Demands

In [ ]:
data_dict_2 = {'NumTowns':[],'Alg_1':[],'Alg_3':[]}

for n in range(max_n):
    for i in range(num_iterations):
        data_dict_2['NumTowns'].append(n)
        demand_means = np.random.uniform(0, demands_max/2, n)
        town_demands, town_expected_demands = make_demands_exponential_distribution(n, demand_means)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict_2['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
#             if j==1:
#                 data_dict['Alg_2'].append(opt - objective(town_demands, waterfilling_online_1(town_expected_demands,town_demands,budget)))
            if j==2:
                data_dict_2['Alg_3'].append(opt - objective(town_demands, waterfilling_dynamic(town_expected_demands,town_demands,budget)))
df_exponential = pd.DataFrame(data_dict_2).melt(id_vars="NumTowns")
fig = px.scatter(df_exponential, x="NumTowns", y="value", color='variable')
fig.show()

### Regret for Uniform Distribution and Fixed Number of Towns

In [ ]:
data_dict_3 = {'BundleSize':[],'Alg_1':[],'Alg_3':[]}

for b in range(1,max_b):
    for i in range(num_iterations):
        data_dict_3['BundleSize'].append(b)
        demand_ranges = np.random.uniform(0, demands_max, fix_num_towns)
        town_demands, town_expected_demands = make_demands_uniform_distribution(fix_num_towns,demand_ranges)
        budget = b

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict_3['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
#             if j==1:
#                 data_dict['Alg_2'].append(opt - objective(town_demands, waterfilling_online_1(town_expected_demands,town_demands,budget)))
            if j==2:
                data_dict_3['Alg_3'].append(opt - objective(town_demands, waterfilling_dynamic(town_expected_demands,town_demands,budget)))
df_uniform_bundle = pd.DataFrame(data_dict_3).melt(id_vars="BundleSize")
fig = px.scatter(df_uniform_bundle, x="BundleSize", y="value", color='variable')
fig.show()

## Least Squares Fitting

In [ ]:
print(np.polyfit(numpy.log(data_dict_1['NumTowns']),data_dict_1['Alg_3'],1))
params = np.polyfit(numpy.log(data_dict_1['NumTowns']),data_dict_1['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_1['NumTowns'])):
    least_squares_score += (a*np.log(data_dict_1['NumTowns'][i]) + b - data_dict_1['Alg_3'])**2
print('Log Score - Uniform Distribution for Demands: ' + str(least_squares_score))

print(np.polyfit(numpy.log(data_dict_2['NumTowns']),data_dict_2['Alg_3'],1))
params = np.polyfit(numpy.log(data_dict_2['NumTowns']),data_dict_2['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_2['NumTowns'])):
    least_squares_score += (a*np.log(data_dict_2['NumTowns'][i]) + b - data_dict_2['Alg_3'])**2
print('Log Score - Exponential Distribution for Demands: ' + str(least_squares_score))

print(np.polyfit(numpy.sqrt(data_dict_1['NumTowns']),data_dict_1['Alg_3'],1))
params = np.polyfit(numpy.sqrt(data_dict_1['NumTowns']),data_dict_1['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_1['NumTowns'])):
    least_squares_score += (a*np.sqrt(data_dict_1['NumTowns'][i]) + b - data_dict_1['Alg_3'])**2
print('Square Root Score - Uniform Distribution for Demands: ' + str(least_squares_score))

print(np.polyfit(numpy.sqrt(data_dict_2['NumTowns']),data_dict_2['Alg_3'],1))
params = np.polyfit(numpy.sqrt(data_dict_2['NumTowns']),data_dict_2['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_2['NumTowns'])):
    least_squares_score += (a*np.sqrt(data_dict_2['NumTowns'][i]) + b - data_dict_2['Alg_3'])**2
print('Square Root Score - Exponential Distribution for Demands: ' + str(least_squares_score))

print(np.polyfit(data_dict_1['NumTowns'],data_dict_1['Alg_3'],1))
params = np.polyfit(numpy.sqrt(data_dict_1['NumTowns']),data_dict_1['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_1['NumTowns'])):
    least_squares_score += (a*data_dict_1['NumTowns'][i] + b - data_dict_1['Alg_3'])**2
print('Linear Score - Uniform Distribution for Demands: ' + str(least_squares_score))

print(np.polyfit(data_dict_2['NumTowns'],data_dict_2['Alg_3'],1))
params = np.polyfit(numpy.sqrt(data_dict_2['NumTowns']),data_dict_2['Alg_3'],1)
a = params[0]
b = params[1]
least_squares_score = 0
for i in range(len(data_dict_2['NumTowns'])):
    least_squares_score += (a*data_dict_2['NumTowns'][i] + b - data_dict_2['Alg_3'])**2
print('Linear Score - Exponential Distribution for Demands: ' + str(least_squares_score))
      